<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

# Model Read

In [17]:
#===============================================
# Basic Imports
BASE_FOLDER = '../../'
%run -i ..\..\utility\feature_extractor\JupyterLoad_feature_extractor.py
%run -i ..\..\utility\modeling\JupyterLoad_modeling.py

from sklearn.metrics import roc_auc_score, confusion_matrix 
from tqdm import tqdm
import os
import glob
import seaborn as sns
sns.set()

load feature_extractor_mother
load feature_extractor_mel_spectra
load feature_extractor_psd
load feature_extractor_ICA2
load feature_extractore_pre_nnFilterDenoise
load extractor_diagram_mother
load Simple_FIR_HP
load TimeSliceAppendActivation
load load_data
Load split_data
Load anomaly_detection_models
Load pseudo_supervised_models
Load tensorflow models
Load detection_pipe


In [18]:
path = BASE_FOLDER + './modeling/unsupervised/models_agg.dataframe'
df = pd.read_pickle(path)

In [23]:
ensemble_pipes = {}
for SNR in ['6dB', 'min6dB']:
    for machine in ['pump', 'fan', 'slider', 'valve']:
        for ID in ['00', '02', '04', '06']:
            paths = glob.glob('pipes/*_{}_{}_ID{}*'.format(SNR, machine, ID))
            if len(paths) !=3:
                print(paths)
            key = '_'.join([SNR, machine, ID])
            ensemble_pipes[key] = []
            for path in paths:
                with open(path, 'rb') as f:
                    ensemble_pipes[key].append(pickle.load(f))

In [25]:
ensemble_dfs = {}
for ensemble_key, ensemble in ensemble_pipes.items():
    for i, pipe in enumerate(ensemble):
        if i==0:
            ensemble_dfs[ensemble_key] = pd.DataFrame(pipe.df_test.groupby(by='path').sum().pred_scores.rename(pipe.model.name+'_'+pipe.task['feat_col']+'_pred_scores').reindex())
            ground_truth = pipe.df_test.groupby(by='path').median().abnormal.rename('ground_truth')
            score_unweighted = pd.Series(np.zeros(ensemble_dfs[ensemble_key].iloc[:,0].shape), index=ensemble_dfs[ensemble_key].index)
            score_weighted = score_unweighted
        else:
            ensemble_dfs[ensemble_key] = ensemble_dfs[ensemble_key].join(pipe.df_test.groupby(by='path').sum().pred_scores.rename(pipe.model.name+'_'+pipe.task['feat_col']+'_pred_scores'))
        
        if pipe.model.name == 'IsoFor':
            if pipe.task['feat_col'] == 'PSD_raw':
                weight = 1.
            else:
                weight = .75
        elif pipe.model.name == 'AutoEnc':
            weight = 1.25
        elif pipe.model.name == 'SVM':
            weight = 1.
        std = ensemble_dfs[ensemble_key].iloc[:,-1].std()
        score_weighted = score_weighted + weight * (ensemble_dfs[ensemble_key].iloc[:,-1] - ensemble_dfs[ensemble_key].iloc[:,-1].mean())/std
        score_unweighted = score_unweighted + (ensemble_dfs[ensemble_key].iloc[:,-1] - ensemble_dfs[ensemble_key].iloc[:,-1].mean())/std
    
    ensemble_dfs[ensemble_key] = ensemble_dfs[ensemble_key].join(score_weighted.rename('score_agg_weighted'))
    std_weighted = ensemble_dfs[ensemble_key].score_agg_weighted.std()
    threshold = -.25*std_weighted
    ensemble_dfs[ensemble_key] = ensemble_dfs[ensemble_key].join(score_unweighted.rename('score_agg_unweighted'))
    ensemble_dfs[ensemble_key] = ensemble_dfs[ensemble_key].join((score_weighted<threshold).rename('prediction_abnormal_weighted').astype(np.float).replace(to_replace=1, value=-1).replace(to_replace=0, value=1))
    ensemble_dfs[ensemble_key] = ensemble_dfs[ensemble_key].join(ground_truth.replace(to_replace=1, value=-1).replace(to_replace=0, value=1))

In [26]:
with open('ensemble_dfs.dataframe', 'wb') as f:
    pickle.dump(ensemble_dfs, f)

In [27]:
results = pd.DataFrame([])
for ensemble_key, ensemble in ensemble_dfs.items():
    keys = ensemble_key.split('_')
    
    conf_mat = confusion_matrix(ensemble.ground_truth, ensemble.prediction_abnormal_weighted)
    
    fp = conf_mat[1, 0].astype(np.float)
    fn = conf_mat[0, 1].astype(np.float)
    tp = conf_mat[0, 0].astype(np.float)
    tn = conf_mat[1, 1].astype(np.float)
    precision = tp/(tp+fp)
    recall = tp/(tp+fn)
    fpr = fp/(fp+tn)
    fnr = fn/(fn+tp)
    r_fp = fp/np.sum(conf_mat)
    r_fn = fn/np.sum(conf_mat)
    
    data = keys + [ conf_mat,
                   fp, fn, tp, tn, precision, recall, fpr, fnr, r_fp, r_fn,
        roc_auc_score(ensemble_dfs[ensemble_key].ground_truth, ensemble_dfs[ensemble_key].score_agg_weighted), 
        roc_auc_score(ensemble_dfs[ensemble_key].ground_truth, ensemble_dfs[ensemble_key].score_agg_unweighted)]

    cols = ['SNR', 'machine', 'ID', 'confusion_matrix', 
            'fp', 'fn', 'tp', 'tn', 'precision', 'recall', 'fpr', 'fnr', 'r_fp', 'r_fn',
            'score_aggr_weighted', 'score_aggr_unweighted']
    
    for pipe in ensemble_pipes[ensemble_key]:
        data.append(pipe.roc_auc)
        cols.append('score_'+pipe.model.name+pipe.task['feat_col'])
    results = results.append(pd.DataFrame(data=[data], columns=cols, index=[ensemble_key]))

In [28]:
results.groupby(by=['SNR','machine']).mean()

fp      fn      tp      tn  precision    recall       fpr  \
SNR    machine                                                                 
6dB    fan      11.75  131.75  237.00  357.00   0.962620  0.639338  0.029499   
       pump      0.00   16.00   98.00  114.00   1.000000  0.863911  0.000000   
       slider    8.75   30.75  191.75  213.75   0.915029  0.812032  0.073034   
       valve     5.25   26.50   93.25  114.50   0.942416  0.778466  0.043855   
min6dB fan      72.75  177.75  191.00  296.00   0.719578  0.521258  0.194473   
       pump     14.00   42.00   72.00  100.00   0.845314  0.636244  0.115695   
       slider   34.00   69.25  153.25  188.50   0.718456  0.591760  0.205758   
       valve    24.25   57.00   62.75   95.50   0.713130  0.523792  0.202574   

                     fnr      r_fp      r_fn  score_aggr_weighted  \
SNR    machine                                                      
6dB    fan      0.360662  0.014750  0.180331             0.977052   
       pump     0.136089  0.000000  0.068045             0.996775   
       slider   0.187968  0.036517  0.093984             0.949757   
       valve    0.221534  0.021928  0.110767             0.956038   
min6dB fan      0.478742  0.097237  0.239371             0.742023   
       pump     0.363756  0.057848  0.181878             0.848067   
       slider   0.408240  0.102879  0.204120             0.746162   
       valve    0.476208  0.101287  0.238104             0.737983   

                score_aggr_unweighted  score_AutoEncMEL_den  \
SNR    machine                                                
6dB    fan                   0.977478              0.939488   
       pump                  0.996666              0.989701   
       slider                0.947578              0.852321   
       valve                 0.953588              0.915167   
min6dB fan                   0.738732              0.746956   
       pump                  0.843473              0.808755   
       slider                0.742950              0.710065   
       valve                 0.731423              0.714022   

                score_IsoForMEL_den  score_IsoForPSD_raw  
SNR    machine                                            
6dB    fan                 0.920700             0.963614  
       pump                0.983836             0.967331  
       slider              0.815524             0.997412  
       valve               0.844289             0.819769  
min6dB fan                 0.699658             0.685163  
       pump                0.742284             0.795610  
       slider              0.669125             0.734730  
       valve               0.631225             0.713538

In [29]:
results.to_pickle('ensemble_results.dataframe')